# Does one-click autopayment enrollment increases user adoption of autopayment and retention after 30 days?

## 1. Load libraries and data

In [21]:
import pandas as pd 
import numpy as np
from scipy.stats import chisquare # for sanity check of randomization
from statsmodels.stats.weightstats  import ttest_ind # for sanity check of randomization
import statsmodels.formula.api as smf 

In [3]:
data = pd.read_csv("C:/Users/VanDoan/OneDrive - Rightpoint/Analysis projects/one-click-feature.csv")
data.head()

,userid,version,monthly_bill,conversion,retention
0,116,oneclick,3,False,False
1,337,oneclick,38,True,False
2,377,old,165,True,False
3,483,old,1,False,False
4,488,old,179,True,True


## 2. Sanity check 
Make sure randomization was done correctly. And if it was not, control for covariates to arrive at reliable findings.

In [9]:
# users count by version
usercount = data.groupby('version')['userid'].nunique()
print(usercount)

version
old         45489
oneclick    44700
Name: userid, dtype: int64


In [10]:
# sanity check if there's a fine balance between treatment and control groups
observations = [usercount['old'], usercount['oneclick']]
expectations = [usercount.sum() / 2, usercount.sum() / 2]

chisquare(observations, expectations)

Power_divergenceResult(statistic=6.9024049496058275, pvalue=0.008607987810836262)

In [11]:
# monthly bill distribution by version
bills = data.groupby('version')['monthly_bill'].describe()
print(bills)

            count       mean         std  min  25%   50%   75%      max
version                                                                
old       45489.0  51.298776  103.294416  0.0  5.0  16.0  52.0   2640.0
oneclick  44700.0  52.456264  256.716423  0.0  5.0  17.0  50.0  49854.0


In [14]:
# sanity check if there's a fine balance between treatment and control groups 
t_stat, p_value, df = ttest_ind(data[data.version == 'old']['monthly_bill'], data[data.version == 'oneclick']['monthly_bill'])

print(t_stat, p_value)

-0.8910426211362967 0.37290868247405207


## 3. Analysis
There is a *confounder*, monthly_bill: Users with high monthly_bill might want to enroll in autopay (with or without the one-click feature) and more likely to stay enrolled. <br>

From sanity check (whether randomization was done correctly), I found an *imbalance* between the treated and controled groups on the confounder.

In [15]:
# outcomes by group: conversion and retention after 30 days
data.groupby('version')['conversion'].mean()

version
old         0.442283
oneclick    0.448188
Name: conversion, dtype: float64

In [16]:
data.groupby('version')['retention'].mean()

version
old         0.182000
oneclick    0.190201
Name: retention, dtype: float64

**Controlling for monthly_bill covariate** <br>
*Note*: Ideally, randomization should do a good job of creating 2 user groups that are similar except for the version they see.

In [18]:
data.head()

,userid,version,monthly_bill,conversion,retention
0,116,oneclick,3,False,False
1,337,oneclick,38,True,False
2,377,old,165,True,False
3,483,old,1,False,False
4,488,old,179,True,True


In [25]:
# creating pseudo population to make the controled and treated groups are comparable
data['version'] = np.where(data['version'] == 'oneclick', 1, 0)

propensity_model = smf.logit('version ~ monthly_bill', data=data).fit(disp = 0)

data = data.assign(propensity_score = propensity_model.predict(data))

data.head()


C:\Users\VanDoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\VanDoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\VanDoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=

,userid,version,monthly_bill,conversion,retention,propensity_score
0,116,0,3,False,False,5.508453e-06
1,337,0,38,True,False,1.186894e-53
2,377,0,165,True,False,1.296212e-226
3,483,0,1,False,False,2.907865e-03
4,488,0,179,True,True,1.111803e-245


In [28]:
ate_conversion = np.mean(data["conversion"] 
        * (data["version"] - data["propensity_score"]) 
        / (data["propensity_score"]*(1 - data["propensity_score"])))

print(ate_conversion)

-0.4201499946747803


In [29]:
ate_retention = np.mean(data["retention"] 
        * (data["version"] - data["propensity_score"]) 
        / (data["propensity_score"]*(1 - data["propensity_score"])))

print(ate_retention)

-0.15129710858551504


## 4. Conclusion
Without controlling for the covariate monthly_bill, the one-click autopayment enrollment seems like increasing conversion rate and retention: <br>
Users seeing the one-click feature converts at 44.8% rate as compared to users seeing the old version (44.2%) <br>
Users seeing the one-click feature retains at 19% rate as compared to users seeing the old version (18.2%)

However, this boost mainly comes from the *difference in the monthly bill distribution between 2 groups*.

**Final conclusion**: the feature does not increace conversion nor retention.